In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_states = "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S"
url_ut = "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U"

state_array = []
for x in range(1, 30):
    if x < 10:
        state_array.append(url_states + "0" + str(x) + ".htm")
    else:
        state_array.append(url_states + str(x) + ".htm")

ut_array = []
for x in range(1, 10):
    ut_array.append(url_ut + "0" + str(x) + ".htm")

all_data = []

state_count = 0
ut_count = 0

def scrape_data(url, is_state=True):
    global state_count, ut_count
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        state_name_element = soup.select_one('html > body > main > div:nth-of-type(1) > h2 > span > strong')
        if state_name_element:
            state_name = state_name_element.text.strip()
        else:
            state_name = "Unknown_State"

        
        table = soup.select_one('div.rslt-table table')
        if table:
            headers = [th.text.strip() for th in table.select('thead th')]
            rows = []
            for tr in table.select('tbody tr'):
                cells = [td.text.strip() for td in tr.select('td')]
                rows.append([state_name] + cells)
            
            
            if not all_data:  
                all_data.append(["State"] + headers)
            all_data.extend(rows)  

            
            if is_state:
                state_count += 1
            else:
                ut_count += 1
        else:
            print("Table not found in URL:", url)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - URL: {url}")
    except Exception as err:
        print(f"Other error occurred: {err} - URL: {url}")

for url in state_array:
    scrape_data(url, is_state=True)

for url in ut_array:
    scrape_data(url, is_state=False)


df = pd.DataFrame(all_data)
df.to_csv("eci_result.csv", index=False, header=False)
print("Data saved to eci_result.csv")

# Print the total number of states and UT
print(f"Total number of states: {state_count}")
print(f"Total number of UTs: {ut_count}")
print(f"Total number of states and UTs: {state_count + ut_count}")


HTTP error occurred: 404 Client Error: Not Found for url: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S09.htm - URL: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S09.htm
HTTP error occurred: 404 Client Error: Not Found for url: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U04.htm - URL: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U04.htm
Data saved to eci_result.csv
Total number of states: 28
Total number of UTs: 8
Total number of states and UTs: 36


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_s01 = "https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm"
url_s18 = "https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm"

def scrape_data(url):
    all_data = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        
        state_name_element = soup.select_one('html > body > main > div:nth-of-type(1) > h2 > span > strong')
        if state_name_element:
            state_name = state_name_element.text.strip()
        else:
            state_name = "Unknown_State"

        
        table = soup.select_one('div.rslt-table table')
        if table:
            headers = [th.text.strip() for th in table.select('thead th')]
            rows = []
            for tr in table.select('tbody tr'):
                cells = [td.text.strip() for td in tr.select('td')]
                rows.append([state_name] + cells)
            
            
            all_data.extend(rows)  
        else:
            print("Table not found in URL:", url)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - URL: {url}")
    except Exception as err:
        print(f"Other error occurred: {err} - URL: {url}")
    
    return all_data


data_s01 = scrape_data(url_s01)
data_s18 = scrape_data(url_s18)


df_s01 = pd.DataFrame(data_s01, columns=["State", "AC No.", "AC Name", "Winner Name", "Party"])
df_s18 = pd.DataFrame(data_s18, columns=["State", "AC No.", "AC Name", "Winner Name", "Party"])

df_s01.to_csv("ap.csv", index=False)
df_s18.to_csv("od.csv", index=False)

print("Data saved to ap.csv and od.csv")


Data saved to ap.csv and od.csv
